In [3]:
%matplotlib inline
from TTS.utils.audio import AudioProcessor
from TTS.utils.visual import plot_spectrogram
from TTS.utils.generic_utils import load_config
import glob 
import IPython.display as ipd
import librosa
import numpy as np
import pandas as pd

In [5]:
config_path = "/home/kurian/Projects/MTTS/project/config.json"
data_path = "/home/kurian/Projects/MTTS/data/slr/female/"
file_paths = glob.glob(data_path + "*.wav", recursive=True)
CONFIG = load_config(config_path)

In [6]:
file_paths

['/home/kurian/Projects/MTTS/data/slr/female/mlf_02321_01935123511.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_09549_02093751751.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_03435_00470895863.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_03132_00881591183.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_04720_00114831578.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_09549_00098782818.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_04720_01536617198.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_09852_01751057952.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_02321_00855181376.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_02925_01272266886.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_05401_01209420986.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_07754_01393111090.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_03132_00800053658.wav',
 '/home/kurian/Projects/MTTS/data/slr/female/mlf_03

In [7]:
file_paths[10]

'/home/kurian/Projects/MTTS/data/slr/female/mlf_05401_01209420986.wav'

### Setup Audio Processor
Play with the AP parameters until you find a good fit with the synthesis speech below. 

In [21]:
audio={
 'audio_processor': 'audio',
 'num_mels': 80,          # In general, you don'tneed to change it 
 'num_freq': 1025,        # In general, you don'tneed to change it 
 'sample_rate': 48000,    # It depends to the sample rate of the dataset.
 'frame_length_ms': 50,   # In general, you don'tneed to change it 
 'frame_shift_ms': 12.5,  # In general, you don'tneed to change it 
 'preemphasis': 0.98,        # In general, 0 gives better voice recovery but makes traning harder. If your model does not train, try 0.97 - 0.99.
 'min_level_db': -100,
 'ref_level_db': 20,      # It is the base DB, higher until you remove the background noise in the spectrogram and then lower until you hear a better speech below.
 'power': 1.5,            # Change this value and listen the synthesized voice. 1.2 - 1.5 are some resonable values.
 'griffin_lim_iters': 60, # It does not give any imporvement for values > 60
 'signal_norm': True,     # This is more about your model. It does not give any change for the synthsis performance.
 'symmetric_norm': False,   # Same as above
 'max_norm': 1,           # Same as above
 'clip_norm': True,       # Same as above
 'mel_fmin': 0.0,        # You can play with this and check mel-spectrogram based voice synthesis below.
 'mel_fmax': 1000.0,        # You can play with this and check mel-spectrogram based voice synthesis below.
 'do_trim_silence': True,
'win_length': 2000} # If you dataset has some silience at the beginning or end, this trims it. Check the AP.load_wav() below,if it causes any difference for the loaded audio file.

AP = AudioProcessor(**audio);

 > Setting up Audio Processor...
 | > sample_rate:48000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:12.5
 | > frame_length_ms:50
 | > ref_level_db:20
 | > num_freq:1025
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:False
 | > mel_fmin:0
 | > mel_fmax:1000.0
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > sound_norm:False
 | > n_fft:2048
 | > hop_length:600
 | > win_length:2400


### Check audio loading 

In [22]:
wav = AP.load_wav(file_paths[10])
ipd.Audio(data=wav, rate=AP.sample_rate) 

In [23]:
len(wav)

305400

In [24]:
wav = AP.load_wav(file_paths[100])
ipd.Audio(data=wav, rate=AP.sample_rate) 

In [25]:
len(wav)

189842

### Generate Mel-Spectrogram and Re-synthesis with GL

In [11]:
mel = AP.melspectrogram(wav)
print("Max:", mel.max())
print("Min:", mel.min())
print("Mean:", mel.mean())
plot_spectrogram(mel.T, AP);

wav_gen = AP.inv_mel_spectrogram(mel)
ipd.Audio(wav_gen, rate=AP.sample_rate)

ParameterError: Target size (2048) must be at least input size (2400)

### Generate Linear-Spectrogram and Re-synthesis with GL

In [14]:
spec = AP.spectrogram(wav)
print("Max:", spec.max())
print("Min:", spec.min())
print("Mean:", spec.mean())
plot_spectrogram(spec.T, AP);

wav_gen = AP.inv_spectrogram(spec)
ipd.Audio(wav_gen, rate=AP.sample_rate)

ParameterError: Target size (2048) must be at least input size (2400)

### Compare values for a certain parameter

Optimize your parameters by comparing different values per parameter at a time.

In [15]:
audio={
 'audio_processor': 'audio',
 'num_mels': 80,          # In general, you don'tneed to change it 
 'num_freq': 1025,        # In general, you don'tneed to change it 
 'sample_rate': 8000,    # It depends to the sample rate of the dataset.
 'frame_length_ms': 50,   # In general, you don'tneed to change it 
 'frame_shift_ms': 12.5,  # In general, you don'tneed to change it 
 'preemphasis': 0.98,        # In general, 0 gives better voice recovery but makes traning harder. If your model does not train, try 0.97 - 0.99.
 'min_level_db': -100,
 'ref_level_db': 20,      # It is the base DB, higher until you remove the background noise in the spectrogram and then lower until you hear a better speech below.
 'power': 1.5,            # Change this value and listen the synthesized voice. 1.2 - 1.5 are some resonable values.
 'griffin_lim_iters': 60, # It does not give any imporvement for values > 60
 'signal_norm': True,     # This is more about your model. It does not give any change for the synthsis performance.
 'symmetric_norm': False,   # Same as above
 'max_norm': 1,           # Same as above
 'clip_norm': True,       # Same as above
 'mel_fmin': 0.0,        # You can play with this and check mel-spectrogram based voice synthesis below.
 'mel_fmax': 8000.0,        # You can play with this and check mel-spectrogram based voice synthesis below.
 'do_trim_silence': True} # If you dataset has some silience at the beginning or end, this trims it. Check the AP.load_wav() below,if it causes any difference for the loaded audio file.

AP = AudioProcessor(**audio);

 > Setting up Audio Processor...
 | > sample_rate:8000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:12.5
 | > frame_length_ms:50
 | > ref_level_db:20
 | > num_freq:1025
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:False
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > sound_norm:False
 | > n_fft:2048
 | > hop_length:100
 | > win_length:400


In [16]:
from librosa import display
from matplotlib import pylab as plt
import IPython
plt.rcParams['figure.figsize'] = (20.0, 16.0)

def compare_values(attribute, values, file):
    """
    attributes (str): the names of the attribute you like to test.
    values (list): list of values to compare.
    file (str): file name to perform the tests.
    """
    wavs = []
    for idx, val in enumerate(values):
        set_val_cmd = "AP.{}={}".format(attribute, val)
        exec(set_val_cmd)
        wav = AP.load_wav(file)
        spec = AP.spectrogram(wav)
        spec_norm = AP._denormalize(spec.T)
        plt.subplot(len(values), 2, 2*idx + 1)
        plt.imshow(spec_norm.T, aspect="auto", origin="lower")
        #         plt.colorbar()
        plt.tight_layout()
        wav_gen = AP.inv_spectrogram(spec)
        wavs.append(wav_gen)
        plt.subplot(len(values), 2, 2*idx + 2)
        display.waveplot(wav, alpha=0.5)
        display.waveplot(wav_gen, alpha=0.25)
        plt.title("{}={}".format(attribute, val))
        plt.tight_layout()
    
    wav = AP.load_wav(file)
    print(" > Ground-truth")
    IPython.display.display(IPython.display.Audio(wav, rate=AP.sample_rate))
    
    for idx, wav_gen in enumerate(wavs):
        val = values[idx]
        print(" > {} = {}".format(attribute, val))
        IPython.display.display(IPython.display.Audio(wav_gen, rate=AP.sample_rate))

In [17]:
compare_values("preemphasis", [0, 0.5, 0.97, 0.98, 0.99], file_paths[10])

AssertionError: 8000 vs 48000

In [18]:
compare_values("ref_level_db", [10, 15, 20, 25, 30, 35, 40], file_paths[10])

AssertionError: 8000 vs 48000